Esta notebook es para ir a airbnb a chekear los precios de los alquileres, porque algunos no eran correctos.

In [1]:
import pandas as pd
import numpy as np
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse
from datetime import datetime
import threading
lock = threading.RLock()
from os import walk

In [2]:
url_proxy = 'https://free-proxy-list.net/anonymous-proxy.html'
response_pxy = requests.get(url_proxy)
pxySoup = BeautifulSoup(response_pxy.text, 'html.parser')
rowsPxy = []
headPxy = []
for tablePart in pxySoup.find_all(attrs={'id':'proxylisttable'})[0].find_all(True, recursive=False):
    if(tablePart.name == 'thead'):
        for row in tablePart.find_all(True, recursive=False):
            headPxy = [tr.text for tr in row]
    if(tablePart.name == 'tbody'):
        for row in tablePart.find_all(True, recursive=False):
            rowsPxy.append([tr.text for tr in row])

proxiesDf = pd.DataFrame(data=rowsPxy, columns=headPxy)
proxiesDf = proxiesDf[(proxiesDf['Anonymity']=='elite proxy')].sort_values(by='Https', ascending=False)
proxiesDf.head()


headersChrome = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}

In [3]:
datasets_names = []
path_df = '../datasets/original_files/'
for (dirpath, dirnames, filenames) in walk(path_df):
    datasets_names.extend(filenames)
    break
dataFrame = False
first = True
for name in datasets_names:
    if(first):
        first = False
        dataFrame = pd.read_csv(path_df+name, compression='gzip')
        dataFrame.set_index('id', drop=False, inplace=True)
        print('adding', len(dataFrame), 'rows')
    else:
        _tmpDf = pd.read_csv(path_df+name, compression='gzip')
        _tmpDf.set_index('id', drop=False, inplace=True)
        print('adding', len(_tmpDf), 'rows')
        dataFrame = pd.concat([dataFrame, _tmpDf])
        del _tmpDf
dataFrame.loc[:,  'price'] = dataFrame.price.apply(lambda x: x[1:-3].replace(',', '')).astype(int)
print('total rows:', len(dataFrame))
dataFrame.head()

/home/santiago/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (43,94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


adding 13927 rows


/home/santiago/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


adding 38481 rows
adding 3799 rows
adding 7211 rows
adding 11668 rows
adding 4868 rows


/home/santiago/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (61,62,94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


adding 50796 rows
total rows: 130750


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,
5570,5570,https://www.airbnb.com/rooms/5570,20200317212944,2020-03-18,Ocean front condo on the sand,Missionbeachretreat. Great Ocean Front Condo...,This charming two bedroom beachfront condomini...,Missionbeachretreat. Great Ocean Front Condo...,none,While some may compare San Diego’s Mission Bea...,...,f,f,strict_14_with_grace_period,f,f,3,3,0,0,NaN
8488,8488,https://www.airbnb.com/rooms/8488,20200317212944,2020-03-18,Juniper Zen Palace- SD's Best Hood!,Walk every where in San Diegos most charming a...,Private craftsman home in the heart of Histori...,Walk every where in San Diegos most charming a...,none,South Park is a charming community and our nei...,...,f,f,strict_14_with_grace_period,t,t,2,2,0,0,0.39
29967,29967,https://www.airbnb.com/rooms/29967,20200317212944,2020-03-18,"Great home, 10 min walk to Beach","This 2 bedroom, 2 full bathroom home offers al...","This two bedroom (1 CA king bed, 1 queen bed),...","This 2 bedroom, 2 full bathroom home offers al...",none,NaN,...,f,f,moderate,f,f,5,5,0,0,0.51
38245,38245,https://www.airbnb.com/rooms/38245,20200317212944,2020-03-18,Point Loma: Den downstairs,Stay in a 1928 Spanish Style home. Den is dow...,Please note: 1928 home; remodeled in 2015. Co...,Stay in a 1928 Spanish Style home. Den is dow...,none,"This neighborhood is an expensive, high-end Po...",...,f,f,moderate,t,t,3,0,3,0,1.23
39516,39516,https://www.airbnb.com/rooms/39516,20200317212944,2020-03-18,Art Studio Retreat/Rustic Cabin,We love having explorers from all over the wor...,"Nestled in South/North Park, a gentrified neig...",We love having explorers from all over the wor...,none,"We love our eclectic neighborhood, with a smal...",...,f,f,strict_14_with_grace_period,t,f,1,1,0,0,3.40


In [14]:
state = 'NY'
targets_df = dataFrame[dataFrame.state == state]
targets = targets_df.index

In [15]:
targets

Int64Index([20183694,     2060,     2595,     3831,     5099,     5114,
                5121,     5178,     5203,     5238,
            ...
            42890182, 42890276, 42890448, 42890474, 42890528, 42890680,
            42890730, 42891018, 42891637, 42892717],
           dtype='int64', name='id', length=50769)

In [16]:
def get_price(id):
    url = 'https://www.airbnb.com.ar/api/v2/pdp_listing_booking_details?_format=for_web_dateless&listing_id='+str(id)+'&currency=USD&guests=1&key=d306zoyjsyarp7ifhu67rjxn52tv0t20&number_of_adults=1'
    response = requests.get(
        url,
        headers=headersChrome
    )
    print()
    data = json.loads(response.text)
    if(data['pdp_listing_booking_details'][0]['p3_display_rate']['currency'] == 'USD'):
        return int(data['pdp_listing_booking_details'][0]['p3_display_rate']['amount'])
    else:
        return None

In [17]:

get_price(20183694)

349

In [18]:
import pickle
real_prices = {}
import time

errors = 0
current = 0

for id in targets:
    current += 1
    if(errors == 25):
        time.sleep(60 * 60)
        errors = 0
    try:
        price = get_price(id)
        real_prices[id] = {
            'price': price,
            'time': time.time()
        }
        if current % 20 == 0:
            pickle.dump( real_prices, open( "../datasets/"+state+"_realPrices.pkl", "wb" ) )
        print('DONE', id)
        errors = 0
    except:
        errors += 1
        real_prices[id] = {
            'price': None,
            'time': time.time()
        }
        print('ERROR', id)
        
pickle.dump( real_prices, open( "../datasets/"+state+"_realPrices.pkl", "wb" ) )


DONE 20183694

DONE 2060

DONE 2595

DONE 3831

DONE 5099

ERROR 5114

DONE 5121

DONE 5178

DONE 5203

DONE 5238

DONE 5441

DONE 5552

DONE 5803

DONE 6021

DONE 6090

DONE 6848

DONE 6990

DONE 7097

DONE 7726

DONE 7750

DONE 7801

DONE 8024

DONE 8110

DONE 8490

DONE 8700

DONE 9657

DONE 9668

DONE 9704

DONE 9783

DONE 10452

DONE 10962

DONE 11708

DONE 11943

DONE 12048

DONE 12192

DONE 12343

DONE 12627

DONE 12937

DONE 12940

DONE 13050

DONE 13394

DONE 13808

ERROR 14287

DONE 14290

DONE 14314

DONE 14322

DONE 14377

ERROR 14468

DONE 14991

DONE 15220

DONE 15341

DONE 15385

DONE 15396

DONE 15711

DONE 15729

DONE 15962

DONE 16326

DONE 16338

DONE 16421

DONE 16458

DONE 16580

ERROR 16595

DONE 16821

DONE 16974

DONE 17037

DONE 17693

DONE 17747

DONE 18127

DONE 18195

DONE 18590

DONE 18728

DONE 19159

ERROR 19169

DONE 19282

DONE 19601

DONE 20300

DONE 20717

DONE 20724

DONE 20734

DONE 20755

ERROR 20853

DONE 20913

DONE 21293

DONE 21456

DONE 21644


DONE 263232

DONE 263502

DONE 263776

DONE 263888

ERROR 264323

DONE 265036

DONE 265064

DONE 265145

DONE 265506

DONE 265831

DONE 266146

DONE 266155

DONE 266437

DONE 266449

DONE 267435

DONE 267535

DONE 267652

DONE 267708

DONE 268362

DONE 268392

DONE 268481

DONE 268868

DONE 269283

DONE 269404

DONE 269889

DONE 270139

DONE 270231

DONE 270315

DONE 270345

DONE 271083

DONE 271128

DONE 271130

DONE 271694

DONE 271954

DONE 272026

DONE 272044

DONE 272427

DONE 272706

DONE 272738

DONE 273190

ERROR 273256

DONE 274062

DONE 274329

DONE 274376

DONE 274743

DONE 274995

DONE 275976

DONE 276216

DONE 276317

ERROR 276482

DONE 276994

ERROR 277207

DONE 277883

DONE 278090

DONE 278145

DONE 278631

DONE 278876

DONE 279857

DONE 280315

DONE 281521

DONE 281851

DONE 282081

DONE 282443

DONE 282514

DONE 282863

DONE 282977

DONE 283072

DONE 283090

DONE 283194

ERROR 283272

DONE 283550

DONE 284208

DONE 284855

ERROR 285442

DONE 286662

DONE 286838

DONE 


DONE 638898

DONE 639199

DONE 639781

DONE 640589

DONE 640691

DONE 640899

DONE 640901

DONE 640908

DONE 640990

DONE 641768

DONE 643877

DONE 643948

DONE 644464

DONE 644575

DONE 644833

DONE 645075

DONE 645693

DONE 645922

DONE 645942

DONE 646391

DONE 646458

DONE 647413

DONE 647520

DONE 647580

ERROR 648047

DONE 648162

DONE 648246

DONE 649561

DONE 649903

DONE 649972

DONE 651375

DONE 651475

DONE 651648

DONE 652371

DONE 652466

DONE 652515

DONE 652648

DONE 652691

DONE 654005

DONE 654232

DONE 654612

DONE 655472

DONE 656281

DONE 657198

DONE 657217

DONE 657727

ERROR 657734

DONE 658008

DONE 658366

DONE 658515

DONE 658709

DONE 658932

DONE 659443

DONE 659880

DONE 659952

DONE 660036

DONE 661814

DONE 663608

ERROR 664047

DONE 665394

DONE 666220

DONE 666315

DONE 666613

DONE 666837

DONE 667375

DONE 667502

DONE 667549

DONE 668400

DONE 668691

ERROR 669633

DONE 670767

DONE 671229

DONE 671633

DONE 672724

DONE 673248

DONE 674692

DONE 67


DONE 978386

DONE 978582

DONE 978615

DONE 980098

DONE 980561

DONE 981494

DONE 982038

DONE 983625

DONE 984721

DONE 985338

DONE 986727

DONE 987329

DONE 987616

DONE 989976

DONE 990130

DONE 990367

DONE 990529

DONE 992508

DONE 992598

DONE 992891

DONE 993040

DONE 993514

DONE 993575

DONE 993576

DONE 995169

DONE 995187

DONE 995215

DONE 995469

DONE 997404

DONE 998423

DONE 999977

DONE 999984

DONE 1000788

DONE 1001147

DONE 1001447

DONE 1002469

DONE 1003754

DONE 1005416

DONE 1005763

ERROR 1009001

DONE 1009883

DONE 1010785

DONE 1011449

DONE 1013648

DONE 1016004

DONE 1016105

DONE 1016352

DONE 1016819

DONE 1018181

DONE 1018529

ERROR 1020625

DONE 1021491

DONE 1021957

DONE 1022204

DONE 1023181

DONE 1023529

DONE 1024113

DONE 1025001

DONE 1025786

DONE 1027808

DONE 1029808

DONE 1030254

DONE 1030392

DONE 1032127

DONE 1032611

DONE 1034347

DONE 1034911

DONE 1036498

DONE 1036734

DONE 1039435

DONE 1039984

DONE 1040156

DONE 1040182

DONE 10


DONE 1617488

DONE 1619695

DONE 1620225

DONE 1621235

DONE 1623431

DONE 1624665

DONE 1628156

DONE 1628564

DONE 1630572

DONE 1631845

DONE 1631938

DONE 1633313

DONE 1633924

ERROR 1634856

DONE 1635088

DONE 1635672

DONE 1640700

DONE 1640995

DONE 1641466

DONE 1644133

DONE 1644146

DONE 1645181

DONE 1645196

DONE 1645667

DONE 1645772

DONE 1645915

DONE 1646432

DONE 1646838

DONE 1646910

DONE 1649957

DONE 1650470

ERROR 1651762

DONE 1652996

DONE 1654551

DONE 1654738

DONE 1654929

DONE 1655550

DONE 1655880

DONE 1656254

DONE 1656539

DONE 1656585

DONE 1656621

DONE 1657110

DONE 1663581

DONE 1664209

DONE 1664614

DONE 1664716

DONE 1665498

DONE 1668328

DONE 1669149

DONE 1669250

DONE 1670517

DONE 1672860

DONE 1672977

DONE 1673040

DONE 1673338

DONE 1677887

DONE 1678809

DONE 1682258

DONE 1682282

DONE 1682451

DONE 1682958

DONE 1682963

DONE 1688711

DONE 1688914

DONE 1690764

DONE 1691188

DONE 1691798

DONE 1693028

DONE 1695731

DONE 1696050

DON

...


DONE 25887913

DONE 25889072

DONE 25891897

DONE 25892102

DONE 25893224

DONE 25893395

DONE 25893890

DONE 25893935

DONE 25894828

DONE 25894924

DONE 25895122

DONE 25895132

ERROR 25895230

DONE 25895519

DONE 25902370

DONE 25903655

DONE 25904988

DONE 25905991

DONE 25909160

DONE 25909161

DONE 25911322

DONE 25912647

ERROR 25912706

DONE 25914515

DONE 25917234

DONE 25917853

DONE 25918623

DONE 25919220

DONE 25921999

DONE 25922009

DONE 25922026

DONE 25925504

DONE 25927829

DONE 25928236

DONE 25934111

DONE 25935054

DONE 25937108

DONE 25937419

DONE 25938492

DONE 25939748

DONE 25940097

DONE 25941089

DONE 25942651

DONE 25943177

DONE 25943635

DONE 25944038

DONE 25944708

DONE 25946015

DONE 25946188

DONE 25946909

DONE 25946924

DONE 25948078

DONE 25948225

DONE 25948509

ERROR 25948511

DONE 25948697

DONE 25948942

DONE 25949359

DONE 25949539

DONE 25949877

DONE 25950590

DONE 25951465

DONE 25961778

DONE 25962847

DONE 25963570

DONE 25965483

DONE 2


DONE 26518547

ERROR 26518779

DONE 26519306

DONE 26519461

DONE 26519832

DONE 26520216

DONE 26522298

DONE 26522398

DONE 26522949

DONE 26524625

ERROR 26526075

DONE 26526269

DONE 26527791

DONE 26528032

DONE 26530921

DONE 26532770

DONE 26533797

DONE 26534838

DONE 26535575

DONE 26538746

DONE 26538883

DONE 26540634

DONE 26550322

DONE 26552340

DONE 26552832

DONE 26552951

DONE 26554732

DONE 26554937

DONE 26555338

DONE 26555450

DONE 26555992

DONE 26557068

DONE 26557266

DONE 26557748

DONE 26557901

DONE 26558970

DONE 26559313

DONE 26559476

DONE 26559892

DONE 26560159

DONE 26560264

DONE 26561464

DONE 26563389

ERROR 26563874

DONE 26564665

DONE 26565864

DONE 26566099

DONE 26566291

DONE 26570172

DONE 26570316

DONE 26571486

DONE 26571706

DONE 26573069

DONE 26574775

ERROR 26576819

DONE 26584254

DONE 26584630

DONE 26587430

DONE 26592537

DONE 26593665

DONE 26594390

DONE 26594677

DONE 26594746

DONE 26595705

DONE 26595733

DONE 26596388

DONE 


DONE 27139801

DONE 27140359

DONE 27140472

DONE 27141013

DONE 27141191

DONE 27141455

DONE 27142225

DONE 27143808

DONE 27149449

DONE 27149630

DONE 27150644

DONE 27154602

DONE 27156174

DONE 27157649

DONE 27159331

DONE 27159664

DONE 27159696

DONE 27160354

DONE 27160938

DONE 27161183

DONE 27161388

ERROR 27162696

DONE 27163003

DONE 27164116

DONE 27164169

DONE 27165508

DONE 27166808

DONE 27166904

DONE 27167258

DONE 27168097

DONE 27169140

DONE 27173512

DONE 27173844

DONE 27188155

DONE 27189113

DONE 27190005

DONE 27190713

DONE 27191858

DONE 27191921

DONE 27192502

DONE 27192776

DONE 27194554

ERROR 27195039

DONE 27195515

DONE 27195566

DONE 27195600

DONE 27195692

DONE 27196488

DONE 27196874

DONE 27197193

DONE 27199508

DONE 27199997

DONE 27200105

DONE 27201242

DONE 27201257

DONE 27201540

DONE 27203333

DONE 27203347

DONE 27203413

DONE 27203779

DONE 27204768

DONE 27205234

DONE 27205506

DONE 27211525

DONE 27214615

DONE 27215273

DONE 27


ERROR 27682933

ERROR 27684910

DONE 27686998

ERROR 27689960

ERROR 27692424

ERROR 27693410

ERROR 27693532

ERROR 27693980

ERROR 27695133

ERROR 27695485

DONE 27695755

ERROR 27696888

DONE 27697284

ERROR 27699052

ERROR 27700332

DONE 27700366

ERROR 27700747

ERROR 27702171

ERROR 27702778

ERROR 27703016

ERROR 27703561

ERROR 27703666

ERROR 27703756

ERROR 27703865

ERROR 27704612

ERROR 27704712

ERROR 27705293

ERROR 27706098

DONE 27706786

ERROR 27706929

ERROR 27707196

ERROR 27707562

ERROR 27707814

ERROR 27710264

ERROR 27712922

ERROR 27714862

ERROR 27720812

ERROR 27722209

ERROR 27723810

ERROR 27724217

ERROR 27724572

ERROR 27725029

ERROR 27725392

ERROR 27725844

ERROR 27727442

ERROR 27728631

ERROR 27730153

DONE 27730191

ERROR 27730757

ERROR 27730786

ERROR 27731269

ERROR 27731372

DONE 27732417

ERROR 27732746

ERROR 27732981

ERROR 27733012

ERROR 27733195

ERROR 27733301

ERROR 27733542

ERROR 27734811

ERROR 27736140

ERROR 27746387

ERROR 27751020

KeyboardInterrupt: 

In [ ]:
pickle.load( open( "../datasets/"+state+"_realPrices.pkl", "rb" ) )

In [ ]:
real_prices

In [ ]:
#dfFinal.loc[list(responses.keys()), 'real_price'] = list(responses.values())

In [ ]:
# dfFinal[['real_price', 'price']][pd.notna(dfFinal.real_price)]

In [ ]:
#dfFinal.to_pickle('../datasets/final_dataset.pkl')